# This source code searches for the ideal number of filters for L and AB branches training with the Cropped-PlantDoc dataset

This raw file supports the paper [Color-aware two-branch DCNN for efficient plant disease classification](https://github.com/joaopauloschuler/two-branch-plant-disease).

You might need to install this on your system:

apt-get install python3-opencv git

In [ ]:
import os

 if not os.path.isdir('k'):
  !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull

!cd k && pip install .

In [ ]:
import cai.layers
import cai.datasets
import cai.models
import cai.densenet
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
import PIL
import cv2
from sklearn.metrics import classification_report

Tensorflow version: 2.4.1
Keras version: 2.4.0


In [ ]:
if not os.path.isdir('up'):
  !git clone https://github.com/joaopauloschuler/PlantDoc-Object-Detection-Dataset.git up

Cloning into 'up'...
remote: Enumerating objects: 5190, done.
remote: Total 5190 (delta 0), reused 0 (delta 0), pack-reused 5190
Receiving objects: 100% (5190/5190), 941.89 MiB | 75.81 MiB/s, done.
Resolving deltas: 100% (2328/2328), done.
Checking out files: 100% (5199/5199), done.


In [ ]:
up_data_dir = os.getcwd()+"/up/TRAIN"
up_test_dir = os.getcwd()+"/up/TEST"
data_dir = "cropped_train/"
test_dir = "cropped_test/"
all_dir = "cropped_all/"

In [ ]:
def CropImages(orig_data_dir, dest_data_dir, csv_file, include_original=False, verbose=True, prefix='img_'):
  # Load the CSV file.
  with open (csv_file, "r") as myfile:
    csv_lines=myfile.readlines()

  # Transform the CSV file into an array  
  line_count = 0
  a_lines = []
  for str_line in csv_lines:
    if line_count > 0: 
      a_line = str_line.replace('\n', '').split(',')
      a_lines.append(a_line)
    line_count = line_count + 1
  #a_lines = np.array(a_lines, dtype=object)
  
  # create destination folder
  if not os.path.isdir(dest_data_dir):
    os.mkdir(dest_data_dir)

  # Save cropped images
  line_count = 0
  failed_count = 0
  original_count = 0
  last_image_file = ''
  for a_line in a_lines:
    dest_folder_name = dest_data_dir+'/'+a_line[3]
    dest_file_name = dest_folder_name+'/'+prefix+str(line_count)+'.jpg'
    orig_file_name = orig_data_dir+'/'+ a_line[0]
    print(line_count,':', a_line, dest_file_name)
    if last_image_file != orig_file_name:
      if os.path.isfile(orig_file_name):
        img = cv2.imread(orig_file_name)
        #cv2_imshow(img)
        last_image_file = orig_file_name
        can_load = True
        original_count = original_count + 1
        if verbose: print("Original imgage shape",img.shape)
        if (include_original):
          if not os.path.isdir(dest_folder_name):
            os.mkdir(dest_folder_name)
          cv2.imwrite(dest_folder_name+'/ori_'+prefix+str(original_count)+'.jpg',img)  
      else:
        if verbose: print('File does not exist:', orig_file_name)
        can_load = False
        #break
    if (can_load):
      y1 = int(a_line[4])
      x1 = int(a_line[5])
      y2 = int(a_line[6])
      x2 = int(a_line[7])

      x_min = np.min([x1, x2])
      x_max = np.max([x1, x2])
      y_min = np.min([y1, y2])
      y_max = np.max([y1, y2])

      crop_img = img[x_min:x_max, y_min:y_max]
      if verbose: print("Cropped imgage shape",crop_img.shape,':', x_min, x_max, y_min, y_max,' deltas:',x_max - x_min, y_max - y_min)
      if not os.path.isdir(dest_folder_name):
        os.mkdir(dest_folder_name)
      if ( (crop_img.shape[0]==0) or (crop_img.shape[1]==0)):
        failed_count = failed_count + 1
        if verbose: print("Failed cropping.");
      else:
        cv2.imwrite(dest_file_name,crop_img)
    else:
      failed_count = failed_count + 1
    line_count = line_count + 1
  print("Processed files:", line_count, "Failed count: ", failed_count, "Original count:", original_count)

In [ ]:
if not os.path.isdir(data_dir):
  CropImages(orig_data_dir=up_data_dir, dest_data_dir=data_dir, csv_file='up/train_labels.csv', include_original=False, prefix='train_')

0 : ['cherry-tree-leaves-and-fruits.jpg', '350', '300', 'Cherry leaf', '198', '77', '299', '252'] cropped_train//Cherry leaf/train_0.jpg
Original imgage shape (300, 350, 3)
Cropped imgage shape (175, 101, 3) : 77 252 198 299  deltas: 175 101
1 : ['cherry-tree-leaves-and-fruits.jpg', '350', '300', 'Cherry leaf', '3', '114', '148', '235'] cropped_train//Cherry leaf/train_1.jpg
Cropped imgage shape (121, 145, 3) : 114 235 3 148  deltas: 121 145
2 : ['cherry-tree-leaves-and-fruits.jpg', '350', '300', 'Cherry leaf', '30', '184', '189', '297'] cropped_train//Cherry leaf/train_2.jpg
Cropped imgage shape (113, 159, 3) : 184 297 30 189  deltas: 113 159
3 : ['cherry-tree-leaves-and-fruits.jpg', '350', '300', 'Cherry leaf', '226', '4', '346', '83'] cropped_train//Cherry leaf/train_3.jpg
Cropped imgage shape (79, 120, 3) : 4 83 226 346  deltas: 79 120
4 : ['peach-and-leaf-stock-image-2809275.jpg', '1300', '1099', 'Peach leaf', '237', '479', '527', '810'] cropped_train//Peach leaf/train_4.jpg
Origi

In [ ]:
if not os.path.isdir(test_dir):
  CropImages(orig_data_dir=up_test_dir, dest_data_dir=test_dir, csv_file='up/test_labels.csv', include_original=False, prefix='test_')

0 : ['Black%20rot%20on%20foliage.jpg', '1060', '795', 'grape leaf black rot', '1', '58', '859', '732'] cropped_test//grape leaf black rot/test_0.jpg
Original imgage shape (795, 1060, 3)
Cropped imgage shape (674, 858, 3) : 58 732 1 859  deltas: 674 858
1 : ['pepper_leaf.jpg', '300', '400', 'Bell_pepper leaf', '40', '58', '251', '392'] cropped_test//Bell_pepper leaf/test_1.jpg
Original imgage shape (400, 300, 3)
Cropped imgage shape (334, 211, 3) : 58 392 40 251  deltas: 334 211
2 : ['depositphotos_1323264-Raspberry-leaf-on-white.jpg', '1024', '772', 'Raspberry leaf', '98', '75', '964', '710'] cropped_test//Raspberry leaf/test_2.jpg
Original imgage shape (772, 1024, 3)
Cropped imgage shape (635, 866, 3) : 75 710 98 964  deltas: 635 866
3 : ['dscn6689.jpg', '1024', '768', 'Tomato mold leaf', '140', '102', '887', '712'] cropped_test//Tomato mold leaf/test_3.jpg
Original imgage shape (768, 1024, 3)
Cropped imgage shape (610, 747, 3) : 102 712 140 887  deltas: 610 747
4 : ['IMG_1246.jpg', '

In [ ]:
print(os.listdir(data_dir))
print(os.listdir(test_dir))
if not os.path.isdir('cropped_test/Potato leaf'):
  !mkdir 'cropped_test/Potato leaf'
if not os.path.isdir('cropped_test/Tomato two spotted spider mites leaf'):
  !mkdir 'cropped_test/Tomato two spotted spider mites leaf'

['grape leaf black rot', 'Apple Scab Leaf', 'Tomato Septoria leaf spot', 'Peach leaf', 'Tomato leaf late blight', 'Potato leaf late blight', 'Potato leaf early blight', 'Tomato leaf bacterial spot', 'Corn Gray leaf spot', 'Bell_pepper leaf', 'Apple rust leaf', 'Strawberry leaf', 'grape leaf', 'Cherry leaf', 'Corn rust leaf', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Potato leaf', 'Bell_pepper leaf spot', 'Squash Powdery mildew leaf', 'Apple leaf', 'Corn leaf blight', 'Raspberry leaf', 'Blueberry leaf', 'Tomato leaf', 'Soyabean leaf', 'Tomato two spotted spider mites leaf', 'Tomato Early blight leaf', 'Tomato leaf mosaic virus']
['grape leaf black rot', 'Apple Scab Leaf', 'Tomato Septoria leaf spot', 'Peach leaf', 'Tomato leaf late blight', 'Potato leaf late blight', 'Potato leaf early blight', 'Tomato leaf bacterial spot', 'Corn Gray leaf spot', 'Bell_pepper leaf', 'Apple rust leaf', 'Strawberry leaf', 'grape leaf', 'Cherry leaf', 'Corn rust leaf', 'Tomato leaf yellow virus', 'T

In [ ]:
if not os.path.isdir(all_dir):
  !mkdir cropped_all
  ! cp -r cropped_test/* cropped_all/
  ! cp -r cropped_train/* cropped_all/

In [ ]:
datagen = cai.util.create_image_generator(vertical_flip=True, rotation_range=90) # width_shift_range=0.5, height_shift_range=0.5

def lrscheduler(epoch):
  return 0.01 * (0.99**epoch)

# Load the dataset again with LAB

In [ ]:
num_classes = 28
batch_size = 32
epochs = 240
target_size_x = 224 # default value is 224
target_size_y = 224 # default value is 224
max_mix_idx = 5

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7,
  root_dir=all_dir, lab=True, 
  verbose=True, bipolar=False, base_model_name='plant_doc',
  has_training=True, has_validation=True, has_testing=True,
  training_size=0.65, validation_size=0.15, test_size=0.2,
  target_size=(target_size_x, target_size_y))

print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

Loading  29  classes.
loading train images
train shape is: (5761, 224, 224, 3)
loading validation images
validation shape is: (1333, 224, 224, 3)
loading test images
test shape is: (1789, 224, 224, 3)
Converting RGB to LAB: 
Converting training.
1000  images converted to lab.
2000  images converted to lab.
3000  images converted to lab.
4000  images converted to lab.
5000  images converted to lab.
Converting validation.
1000  images converted to lab.
Converting test.
1000  images converted to lab.
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.11655408  max: 0.9557502
Channel  2  min: 1.0877848e-05  max: 0.9704958
Loaded.
(5761, 224, 224, 3) (1333, 224, 224, 3) (1789, 224, 224, 3)
(5761, 29) (1333, 29) (1789, 29)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28], y=[ 0  0  0 ... 28 28 28] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [ ]:
def work_on_inception_v3(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  deep_two_paths_compression=0.6
  for l_ratio in [-1.0, 0.0, 0.2, 0.5, 0.8, 1.0]:
      if l_ratio < 0:
          two_paths_first_block=False
          two_paths_second_block=False
          deep_two_paths=False
      else:
          two_paths_first_block=True
          two_paths_second_block=False
          deep_two_paths=False
      basefilename = 'JP27B13-InceptionV3-CroppedPlantDoc-'+str(l_ratio)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      model = cai.models.compiled_two_path_inception_v3(
        input_shape=input_shape,
        classes=29, 
        two_paths_first_block=two_paths_first_block,
        two_paths_second_block=two_paths_second_block,
        deep_two_paths=deep_two_paths,
        deep_two_paths_compression=deep_two_paths_compression,
        l_ratio=l_ratio,
        ab_ratio=(1-l_ratio),
        max_mix_idx=max_mix_idx, 
        model_name='two_path_inception_v3_mixed5'
        )

      if (show_model): model.summary()

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        history = model.fit(
          x = datagen.flow(train_x, train_y, batch_size=batch_size),
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(val_x, val_y),
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(lrscheduler)], # cai.densenet.cyclical_smooth_lrscheduler
          workers=max([multiprocessing.cpu_count(), 4]) # this option 
          )
      if (test_results):
        print('Best Model Results: '+basefilename)
        model = keras.models.load_model(best_result_file_name, custom_objects={'CopyChannels': cai.layers.CopyChannels})
        evaluated = model.evaluate(test_x,test_y)
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(test_x)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(test_y, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [ ]:
work_on_inception_v3(show_model=True, run_fit=False, test_results=False)

Running: JP27B13-InceptionV3-CroppedPlantDoc--1.0
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
____________________________________________

# Fitting

In [ ]:
work_on_inception_v3(show_model=False, run_fit=True, test_results=True)

Running: JP27B13-InceptionV3-CroppedPlantDoc--1.0
Epoch 1/240
181/181 [==============================] - 231s 191ms/step - loss: 3.0444 - accuracy: 0.1547 - val_loss: 3.4535 - val_accuracy: 0.0878

Epoch 00001: val_accuracy improved from -inf to 0.08777, saving model to JP27B13-InceptionV3-CroppedPlantDoc--1.0-best_result.hdf5
Epoch 2/240
181/181 [==============================] - 32s 175ms/step - loss: 2.7514 - accuracy: 0.2091 - val_loss: 3.1781 - val_accuracy: 0.1230

Epoch 00002: val_accuracy improved from 0.08777 to 0.12303, saving model to JP27B13-InceptionV3-CroppedPlantDoc--1.0-best_result.hdf5
Epoch 3/240
181/181 [==============================] - 32s 174ms/step - loss: 2.5478 - accuracy: 0.2498 - val_loss: 2.8917 - val_accuracy: 0.1973

Epoch 00003: val_accuracy improved from 0.12303 to 0.19730, saving model to JP27B13-InceptionV3-CroppedPlantDoc--1.0-best_result.hdf5
Epoch 4/240
181/181 [==============================] - 32s 173ms/step - loss: 2.4023 - accuracy: 0.2934 - val

# Test Results

In [ ]:
work_on_inception_v3(show_model=False, run_fit=False, test_results=True)

Running: JP27B13-InceptionV3-CroppedPlantDoc--1.0
Best Model Results: JP27B13-InceptionV3-CroppedPlantDoc--1.0
56/56 [==============================] - 3s 43ms/step - loss: 1.2601 - accuracy: 0.7602
loss 1.260094404220581
acc 0.7602012157440186
Finished: JP27B13-InceptionV3-CroppedPlantDoc--1.0
Running: JP27B13-InceptionV3-CroppedPlantDoc-0.0
Best Model Results: JP27B13-InceptionV3-CroppedPlantDoc-0.0
56/56 [==============================] - 3s 43ms/step - loss: 1.5187 - accuracy: 0.7155
loss 1.5187485218048096
acc 0.7154834866523743
Finished: JP27B13-InceptionV3-CroppedPlantDoc-0.0
Running: JP27B13-InceptionV3-CroppedPlantDoc-0.2
Best Model Results: JP27B13-InceptionV3-CroppedPlantDoc-0.2
56/56 [==============================] - 3s 42ms/step - loss: 1.2810 - accuracy: 0.7658
loss 1.2809736728668213
acc 0.7657909393310547
Finished: JP27B13-InceptionV3-CroppedPlantDoc-0.2
Running: JP27B13-InceptionV3-CroppedPlantDoc-0.5
Best Model Results: JP27B13-InceptionV3-CroppedPlantDoc-0.5
56/56 [

# Calculate F1

In [ ]:
work_on_inception_v3(show_model=False, run_fit=False, test_results=True, calc_f1=True)

Running: JP27B13-InceptionV3-CroppedPlantDoc--1.0
Best Model Results: JP27B13-InceptionV3-CroppedPlantDoc--1.0
56/56 [==============================] - 3s 43ms/step - loss: 1.2601 - accuracy: 0.7602
loss 1.260094404220581
acc 0.7602012157440186
Predicted Shape: (1789, 29)
Pred classes shape: (1789,)
Test classes shape: (1789,)
              precision    recall  f1-score   support

           0     0.4545    0.4286    0.4412        35
           1     0.7647    0.7800    0.7723        50
           2     0.7778    0.7778    0.7778        36
           3     0.8182    0.6923    0.7500        65
           4     0.6786    0.7170    0.6972        53
           5     0.8807    0.9172    0.8986       169
           6     0.7143    0.5208    0.6024        48
           7     0.3333    0.1875    0.2400        16
           8     0.7733    0.7838    0.7785        74
           9     0.7692    0.7692    0.7692        26
          10     0.8175    0.9032    0.8582       124
          11     0.666

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best Model Results: JP27B13-InceptionV3-CroppedPlantDoc-0.0
56/56 [==============================] - 3s 43ms/step - loss: 1.5187 - accuracy: 0.7155
loss 1.5187485218048096
acc 0.7154834866523743
Predicted Shape: (1789, 29)
Pred classes shape: (1789,)
Test classes shape: (1789,)
              precision    recall  f1-score   support

           0     0.6923    0.2571    0.3750        35
           1     0.7609    0.7000    0.7292        50
           2     0.6429    0.7500    0.6923        36
           3     0.7458    0.6769    0.7097        65
           4     0.6087    0.5283    0.5657        53
           5     0.8412    0.8462    0.8437       169
           6     0.6842    0.5417    0.6047        48
           7     0.4167    0.3125    0.3571        16
           8     0.7250    0.7838    0.7532        74
           9     0.5806    0.6923    0.6316        26
          10     0.7744    0.8306    0.8016       124
          11     1.0000    0.3333    0.5000         3
          12     0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best Model Results: JP27B13-InceptionV3-CroppedPlantDoc-0.2
56/56 [==============================] - 3s 42ms/step - loss: 1.2810 - accuracy: 0.7658
loss 1.2809736728668213
acc 0.7657909393310547
Predicted Shape: (1789, 29)
Pred classes shape: (1789,)
Test classes shape: (1789,)
              precision    recall  f1-score   support

           0     0.6538    0.4857    0.5574        35
           1     0.7547    0.8000    0.7767        50
           2     0.9091    0.8333    0.8696        36
           3     0.7541    0.7077    0.7302        65
           4     0.7647    0.7358    0.7500        53
           5     0.8564    0.9172    0.8857       169
           6     0.7027    0.5417    0.6118        48
           7     0.4667    0.4375    0.4516        16
           8     0.7722    0.8243    0.7974        74
           9     0.7778    0.8077    0.7925        26
          10     0.8102    0.8952    0.8506       124
          11     0.5000    0.3333    0.4000         3
          12     0

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best Model Results: JP27B13-InceptionV3-CroppedPlantDoc-0.8
56/56 [==============================] - 3s 42ms/step - loss: 1.2939 - accuracy: 0.7585
loss 1.2939047813415527
acc 0.7585242986679077
Predicted Shape: (1789, 29)
Pred classes shape: (1789,)
Test classes shape: (1789,)
              precision    recall  f1-score   support

           0     0.5143    0.5143    0.5143        35
           1     0.7800    0.7800    0.7800        50
           2     0.7576    0.6944    0.7246        36
           3     0.8033    0.7538    0.7778        65
           4     0.7170    0.7170    0.7170        53
           5     0.9157    0.8994    0.9075       169
           6     0.7179    0.5833    0.6437        48
           7     0.4286    0.3750    0.4000        16
           8     0.7229    0.8108    0.7643        74
           9     0.6897    0.7692    0.7273        26
          10     0.8462    0.8871    0.8661       124
          11     0.5000    0.3333    0.4000         3
          12     0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best Model Results: JP27B13-InceptionV3-CroppedPlantDoc-1.0
56/56 [==============================] - 3s 44ms/step - loss: 1.9974 - accuracy: 0.6467
loss 1.9974042177200317
acc 0.6467300057411194
Predicted Shape: (1789, 29)
Pred classes shape: (1789,)
Test classes shape: (1789,)
              precision    recall  f1-score   support

           0     0.3617    0.4857    0.4146        35
           1     0.6364    0.7000    0.6667        50
           2     0.5000    0.3611    0.4194        36
           3     0.6610    0.6000    0.6290        65
           4     0.6667    0.4151    0.5116        53
           5     0.7988    0.7751    0.7868       169
           6     0.4035    0.4792    0.4381        48
           7     0.2857    0.1250    0.1739        16
           8     0.7024    0.7973    0.7468        74
           9     0.7308    0.7308    0.7308        26
          10     0.7172    0.8387    0.7732       124
          11     0.5000    0.3333    0.4000         3
          12     0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
